<a href="https://colab.research.google.com/github/Ingenjoy/Recommendation-System/blob/master/wide_and_deep_%EC%83%81%EC%84%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np

from keras.models import Sequential, Model
from keras.layers import Input, Dense, concatenate
from keras.optimizers import Adam
import easydict


# feature로 이용할 컬럼 정의
COLUMNS = ["age", "workclass", "fnlwgt", "education", "education_num", "marital_status",
          "occupation", "relationship", "race", "gender", "capital_gain", "capital_loss", "hours_per_week", "native_country", "income_bracket"]
# ?          
LABEL_COLUMN = "label"
# 범주형 Feature
CATEGORICAL_COLUMNS = ["workclass", "education", "marital_status", "occupation", "relationship", "race", "gender", "native_country"]
# 숫자형 Feature
NUMERICAL_COLUMNS = ["age", "education_num", "capital_gain", "capital_loss", "hours_per_week"]


In [0]:
# wide 모델
class Wide:
    def __init__(self, args):
        self.learning_rate = args.learning_rate
        self.epochs = args.epochs
        self.batch_size = args.batch_size
        self.input_dim = args.input_dim
        self.model = self.classifier()

    def classifier(self):
        model = Sequential()
        model.add(Dense(1, activation='sigmoid', input_dim=self.input_dim))
        optimizer = Adam(lr=self.learning_rate, beta_1=0.9, beta_2=0.999)
        model.compile(loss='binary_crossentropy',
                      optimizer=optimizer,
                      metrics=['accuracy'])
        return model

    def fit(self, x, y):
      # validation_split : validation set 20%
        self.model.fit(x, y, epochs=self.epochs, batch_size=self.batch_size, validation_split=0.2)

    def print_performance(self, x, y):
        performance_test = self.model.evaluate(x, y, batch_size=self.batch_size)
        print('Test Loss and Accuracy ->', performance_test)

In [0]:
# Deeo 모델
class Deep:
    def __init__(self, args):
        self.learning_rate = args.learning_rate
        self.epochs = args.epochs
        self.batch_size = args.batch_size
        self.input_dim = args.input_dim
        self.model = self.classifier()

    def classifier(self):
        model = Sequential()
        model.add(Dense(100, activation='relu', input_dim=self.input_dim))
        model.add(Dense(50, activation='relu'))
        model.add(Dense(1, activation='sigmoid'))

        optimizer = Adam(lr=self.learning_rate, beta_1=0.9, beta_2=0.999)
        model.compile(loss='binary_crossentropy',
                      optimizer=optimizer,
                      metrics=['accuracy'])
        return model

    def fit(self, x, y):
        self.model.fit(x, y, epochs=self.epochs, batch_size=self.batch_size, validation_split=0.2)

    def print_performance(self, x, y):
        performance_test = self.model.evaluate(x, y, batch_size=self.batch_size)
        print('Test Loss and Accuracy ->', performance_test)


In [0]:
class WideAndDeep:
    def __init__(self, args):
        self.learning_rate = args.learning_rate
        self.epochs = args.epochs
        self.batch_size = args.batch_size
        self.input_dim = args.input_dim
        self.model = self.classifier()

    def classifier(self):
        optimizer = Adam(lr=self.learning_rate, beta_1=0.9, beta_2=0.999)

        # wide part
        wide = Input(shape=(self.input_dim,))

        # deep part
        deep_input = Input(shape=(self.input_dim,))
        deep = Dense(100, activation='relu')(deep_input)
        deep = Dense(50, activation='relu')(deep)

        # concatenate : wide and deep
        wide_n_deep = concatenate([wide, deep])
        wide_n_deep = Dense(1, activation='sigmoid')(wide_n_deep)
        model = Model(inputs=[wide, deep_input], outputs=wide_n_deep)
        model.compile(loss='binary_crossentropy',
                      optimizer=optimizer,
                      metrics=['accuracy'])
        return model

    def fit(self, wide_x, deep_x, y):
        self.model.fit([wide_x, deep_x], y, epochs=self.epochs, batch_size=self.batch_size, validation_split=0.2)

    def print_performance(self, wide_x, deep_x, y):
        performance_test = self.model.evaluate([wide_x, deep_x], y, batch_size=self.batch_size)
        print('Test Loss and Accuracy ->', performance_test)

In [0]:
def main(model_param):
    # prepare dataset
    df_train = pd.read_csv('http://mlr.cs.umass.edu/ml/machine-learning-databases/adult/adult.data',
                           header=None,
                           names=COLUMNS,
                           skipinitialspace=True)
    df_test = pd.read_csv('http://mlr.cs.umass.edu/ml/machine-learning-databases/adult/adult.test',
                          header=None,
                          names=COLUMNS,
                          skipinitialspace=True,
                          skiprows=1)
    df = pd.concat([df_train, df_test])
    df[LABEL_COLUMN] = df['income_bracket'].apply(lambda x: ">50K" in x).astype(int)
    y = df[LABEL_COLUMN].values
    df.pop(LABEL_COLUMN)
    df.pop("income_bracket")
    df = pd.get_dummies(df, columns=[x for x in CATEGORICAL_COLUMNS])
    df = pd.DataFrame(MinMaxScaler().fit_transform(df), columns=df.columns)
    x = df.values

    # split train, test
    train_length = len(df_train)
    x_train = x[:train_length]
    y_train = y[:train_length]
    x_test = x[train_length:]
    y_test = y[train_length:]

    args = easydict.EasyDict({
        "batch_size": 500,
        "epochs": 30,
        "learning_rate": 0.001,
        "input_dim": x_train.shape[1]
    })

    if model_param == "deep":
        deep = Deep(args)
        deep.fit(x_train, y_train)
        deep.print_performance(x_test, y_test)
    elif model_param == 'wide':
        wide = Wide(args)
        wide.fit(x_train, y_train)
        wide.print_performance(x_test, y_test)
    else:
        wide_n_deep = WideAndDeep(args)
        wide_n_deep.fit(x_train, x_train, y_train)
        wide_n_deep.print_performance(x_test, x_test, y_test)

        # prediction for individual
        x_predict_test = x_test[np.newaxis, 0, :]
        y_predict_test = y_test[0]
        result = wide_n_deep.model.predict([x_predict_test, x_predict_test])
        print('result predicted:', result[0][0])
        print('result predicted:', round(result[0][0]))
        print('result real:', y_predict_test)

In [0]:
if __name__ == '__main__':
    main('deep')

    # score (test loss and acc)
    # 1. wide : [0.34640224496809097, 0.8394447512213981]
    # 2. deep : [0.3533135050656381, 0.8403660668860226]
    # 3. wide and deep : [0.352460421507851, 0.8432528705885703]

In [0]:
if __name__ == '__main__':
    main('wide')

    # score (test loss and acc)
    # 1. wide : [0.34640224496809097, 0.8394447512213981]
    # 2. deep : [0.3533135050656381, 0.8403660668860226]
    # 3. wide and deep : [0.352460421507851, 0.8432528705885703]

In [0]:
if __name__ == '__main__':
    main('widendeep')

    # score (test loss and acc)
    # 1. wide : [0.34640224496809097, 0.8394447512213981]
    # 2. deep : [0.3533135050656381, 0.8403660668860226]
    # 3. wide and deep : [0.352460421507851, 0.8432528705885703]

In [0]:
  # prepare dataset
  df_train = pd.read_csv('http://mlr.cs.umass.edu/ml/machine-learning-databases/adult/adult.data',
                          header=None,
                          names=COLUMNS,
                          skipinitialspace=True)
  df_test = pd.read_csv('http://mlr.cs.umass.edu/ml/machine-learning-databases/adult/adult.test',
                        header=None,
                        names=COLUMNS,
                        skipinitialspace=True,
                        skiprows=1)
  df = pd.concat([df_train, df_test])

In [0]:
print("train : "+str(df_train.shape))
print("test : "+str(df_test.shape))
print("train+test : "+str(df.shape))

train : (32561, 15)
test : (16281, 15)
train+test : (48842, 15)


In [0]:
df[LABEL_COLUMN] = df['income_bracket'].apply(lambda x: ">50K" in x).astype(int)

In [0]:
df.shape

(48842, 16)

In [0]:
df[:10]

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income_bracket,label
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K,0
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K,0
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K,0
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K,0
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K,0
5,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K,0
6,49,Private,160187,9th,5,Married-spouse-absent,Other-service,Not-in-family,Black,Female,0,0,16,Jamaica,<=50K,0
7,52,Self-emp-not-inc,209642,HS-grad,9,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,45,United-States,>50K,1
8,31,Private,45781,Masters,14,Never-married,Prof-specialty,Not-in-family,White,Female,14084,0,50,United-States,>50K,1
9,42,Private,159449,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,5178,0,40,United-States,>50K,1


In [0]:
# 정답을 y 에 입력
# label, income_bracket 컬럼 삭제
y = df[LABEL_COLUMN].values
df.pop(LABEL_COLUMN)
df.pop("income_bracket")

0         <=50K
1         <=50K
2         <=50K
3         <=50K
4         <=50K
          ...  
16276    <=50K.
16277    <=50K.
16278    <=50K.
16279    <=50K.
16280     >50K.
Name: income_bracket, Length: 48842, dtype: object

In [0]:
# categorical 컬럼을 dummy 변수로 변환
df = pd.get_dummies(df, columns=[x for x in CATEGORICAL_COLUMNS])

In [0]:
df.shape

(48842, 108)

In [0]:
df[:10]

,age,fnlwgt,education_num,capital_gain,capital_loss,hours_per_week,workclass_?,workclass_Federal-gov,workclass_Local-gov,workclass_Never-worked,workclass_Private,workclass_Self-emp-inc,workclass_Self-emp-not-inc,workclass_State-gov,workclass_Without-pay,education_10th,education_11th,education_12th,education_1st-4th,education_5th-6th,education_7th-8th,education_9th,education_Assoc-acdm,education_Assoc-voc,education_Bachelors,education_Doctorate,education_HS-grad,education_Masters,education_Preschool,education_Prof-school,education_Some-college,marital_status_Divorced,marital_status_Married-AF-spouse,marital_status_Married-civ-spouse,marital_status_Married-spouse-absent,marital_status_Never-married,marital_status_Separated,marital_status_Widowed,occupation_?,occupation_Adm-clerical,...,native_country_Canada,native_country_China,native_country_Columbia,native_country_Cuba,native_country_Dominican-Republic,native_country_Ecuador,native_country_El-Salvador,native_country_England,native_country_France,native_country_Germany,native_country_Greece,native_country_Guatemala,native_country_Haiti,native_country_Holand-Netherlands,native_country_Honduras,native_country_Hong,native_country_Hungary,native_country_India,native_country_Iran,native_country_Ireland,native_country_Italy,native_country_Jamaica,native_country_Japan,native_country_Laos,native_country_Mexico,native_country_Nicaragua,native_country_Outlying-US(Guam-USVI-etc),native_country_Peru,native_country_Philippines,native_country_Poland,native_country_Portugal,native_country_Puerto-Rico,native_country_Scotland,native_country_South,native_country_Taiwan,native_country_Thailand,native_country_Trinadad&Tobago,native_country_United-States,native_country_Vietnam,native_country_Yugoslavia
0,39,77516,13,2174,0,40,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,50,83311,13,0,0,13,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,38,215646,9,0,0,40,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,53,234721,7,0,0,40,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,28,338409,13,0,0,40,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,37,284582,14,0,0,40,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
6,49,160187,5,0,0,16,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,52,209642,9,0,0,45,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
8,31,45781,14,14084,0,50,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
9,42,159449,13,5178,0,40,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [0]:
# 데이터를 0~1로 스케일링
df = pd.DataFrame(MinMaxScaler().fit_transform(df), columns=df.columns)

In [0]:
df[:10]

,age,fnlwgt,education_num,capital_gain,capital_loss,hours_per_week,workclass_?,workclass_Federal-gov,workclass_Local-gov,workclass_Never-worked,workclass_Private,workclass_Self-emp-inc,workclass_Self-emp-not-inc,workclass_State-gov,workclass_Without-pay,education_10th,education_11th,education_12th,education_1st-4th,education_5th-6th,education_7th-8th,education_9th,education_Assoc-acdm,education_Assoc-voc,education_Bachelors,education_Doctorate,education_HS-grad,education_Masters,education_Preschool,education_Prof-school,education_Some-college,marital_status_Divorced,marital_status_Married-AF-spouse,marital_status_Married-civ-spouse,marital_status_Married-spouse-absent,marital_status_Never-married,marital_status_Separated,marital_status_Widowed,occupation_?,occupation_Adm-clerical,...,native_country_Canada,native_country_China,native_country_Columbia,native_country_Cuba,native_country_Dominican-Republic,native_country_Ecuador,native_country_El-Salvador,native_country_England,native_country_France,native_country_Germany,native_country_Greece,native_country_Guatemala,native_country_Haiti,native_country_Holand-Netherlands,native_country_Honduras,native_country_Hong,native_country_Hungary,native_country_India,native_country_Iran,native_country_Ireland,native_country_Italy,native_country_Jamaica,native_country_Japan,native_country_Laos,native_country_Mexico,native_country_Nicaragua,native_country_Outlying-US(Guam-USVI-etc),native_country_Peru,native_country_Philippines,native_country_Poland,native_country_Portugal,native_country_Puerto-Rico,native_country_Scotland,native_country_South,native_country_Taiwan,native_country_Thailand,native_country_Trinadad&Tobago,native_country_United-States,native_country_Vietnam,native_country_Yugoslavia
0,0.301370,0.044131,0.800000,0.021740,0.0,0.397959,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.452055,0.048052,0.800000,0.000000,0.0,0.122449,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.287671,0.137581,0.533333,0.000000,0.0,0.397959,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.493151,0.150486,0.400000,0.000000,0.0,0.397959,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.150685,0.220635,0.800000,0.000000,0.0,0.397959,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.273973,0.184219,0.866667,0.000000,0.0,0.397959,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
6,0.438356,0.100061,0.266667,0.000000,0.0,0.153061,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.

In [0]:
x = df.values
x

array([[0.30136986, 0.04413121, 0.8       , ..., 1.        , 0.        ,
        0.        ],
       [0.45205479, 0.04805174, 0.8       , ..., 1.        , 0.        ,
        0.        ],
       [0.28767123, 0.13758131, 0.53333333, ..., 1.        , 0.        ,
        0.        ],
       ...,
       [0.28767123, 0.24537874, 0.8       , ..., 1.        , 0.        ,
        0.        ],
       [0.36986301, 0.04844413, 0.8       , ..., 1.        , 0.        ,
        0.        ],
       [0.24657534, 0.11491866, 0.8       , ..., 1.        , 0.        ,
        0.        ]])

In [0]:
x.shape

(48842, 108)

In [0]:
type(x)

numpy.ndarray

In [0]:
  # split train, test
  train_length = len(df_train)
  x_train = x[:train_length]
  y_train = y[:train_length]
  x_test = x[train_length:]
  y_test = y[train_length:]

In [0]:
  args = easydict.EasyDict({
    "batch_size": 500,
    "epochs": 30,
    "learning_rate": 0.001,
    "input_dim": x_train.shape[1]
  })

In [0]:
wide = Wide(args)
wide.fit(x_train, y_train)
wide.print_performance(x_test, y_test)

Train on 26048 samples, validate on 6513 samples
Epoch 1/30
26048/26048 [==============================] - 0s 7us/step - loss: 0.5407 - accuracy: 0.7544 - val_loss: 0.5190 - val_accuracy: 0.7530
Epoch 2/30
26048/26048 [==============================] - 0s 5us/step - loss: 0.4997 - accuracy: 0.7592 - val_loss: 0.4883 - val_accuracy: 0.7556
Epoch 3/30
26048/26048 [==============================] - 0s 5us/step - loss: 0.4736 - accuracy: 0.7637 - val_loss: 0.4659 - val_accuracy: 0.7606
Epoch 4/30
26048/26048 [==============================] - 0s 5us/step - loss: 0.4539 - accuracy: 0.7731 - val_loss: 0.4482 - val_accuracy: 0.7803
Epoch 5/30
26048/26048 [==============================] - 0s 5us/step - loss: 0.4384 - accuracy: 0.7948 - val_loss: 0.4343 - val_accuracy: 0.8032
Epoch 6/30
26048/26048 [==============================] - 0s 5us/step - loss: 0.4261 - accuracy: 0.8095 - val_loss: 0.4232 - val_accuracy: 0.8119
Epoch 7/30
26048/26048 [==============================] - 0s 5us/step - los

In [0]:
deep = Deep(args)
deep.fit(x_train, y_train)
deep.print_performance(x_test, y_test)

Train on 26048 samples, validate on 6513 samples
Epoch 1/30
26048/26048 [==============================] - 0s 13us/step - loss: 0.4567 - accuracy: 0.7848 - val_loss: 0.3759 - val_accuracy: 0.8260
Epoch 2/30
26048/26048 [==============================] - 0s 8us/step - loss: 0.3593 - accuracy: 0.8336 - val_loss: 0.3508 - val_accuracy: 0.8345
Epoch 3/30
26048/26048 [==============================] - 0s 9us/step - loss: 0.3468 - accuracy: 0.8392 - val_loss: 0.3469 - val_accuracy: 0.8363
Epoch 4/30
26048/26048 [==============================] - 0s 9us/step - loss: 0.3385 - accuracy: 0.8421 - val_loss: 0.3375 - val_accuracy: 0.8406
Epoch 5/30
26048/26048 [==============================] - 0s 8us/step - loss: 0.3322 - accuracy: 0.8444 - val_loss: 0.3324 - val_accuracy: 0.8449
Epoch 6/30
26048/26048 [==============================] - 0s 8us/step - loss: 0.3270 - accuracy: 0.8477 - val_loss: 0.3309 - val_accuracy: 0.8466
Epoch 7/30
26048/26048 [==============================] - 0s 9us/step - lo

In [0]:
wide_n_deep = WideAndDeep(args)
wide_n_deep.fit(x_train, x_train, y_train)
wide_n_deep.print_performance(x_test, x_test, y_test)

Train on 26048 samples, validate on 6513 samples
Epoch 1/30
26048/26048 [==============================] - 0s 14us/step - loss: 0.5013 - accuracy: 0.7585 - val_loss: 0.3885 - val_accuracy: 0.8250
Epoch 2/30
26048/26048 [==============================] - 0s 10us/step - loss: 0.3649 - accuracy: 0.8310 - val_loss: 0.3527 - val_accuracy: 0.8369
Epoch 3/30
26048/26048 [==============================] - 0s 11us/step - loss: 0.3469 - accuracy: 0.8372 - val_loss: 0.3419 - val_accuracy: 0.8415
Epoch 4/30
26048/26048 [==============================] - 0s 10us/step - loss: 0.3394 - accuracy: 0.8406 - val_loss: 0.3365 - val_accuracy: 0.8439
Epoch 5/30
26048/26048 [==============================] - 0s 10us/step - loss: 0.3342 - accuracy: 0.8431 - val_loss: 0.3332 - val_accuracy: 0.8466
Epoch 6/30
26048/26048 [==============================] - 0s 10us/step - loss: 0.3292 - accuracy: 0.8459 - val_loss: 0.3306 - val_accuracy: 0.8440
Epoch 7/30
26048/26048 [==============================] - 0s 10us/ste

In [0]:
x_test.shape

(16281, 108)

In [0]:
# prediction for individual
x_predict_test = x_test[np.newaxis, 0, :]
y_predict_test = y_test[0]

In [0]:
y_test[0]

0

In [0]:
x_predict_test.shape

(1, 108)

In [0]:
x_predict_test

array([[0.10958904, 0.14512876, 0.4       , 0.        , 0.        ,
        0.39795918, 0.        , 0.        , 0.        , 0.        ,
        1.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 1.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        1.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        1.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 1.        , 0.        , 0.        , 0.        ,
        0.        , 1.        , 0.        , 0.        , 0.        ,
        1.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.  

In [0]:
x_test[0, :]

array([0.10958904, 0.14512876, 0.4       , 0.        , 0.        ,
       0.39795918, 0.        , 0.        , 0.        , 0.        ,
       1.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 1.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       1.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       1.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 1.        , 0.        , 0.        , 0.        ,
       0.        , 1.        , 0.        , 0.        , 0.        ,
       1.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

In [0]:
x_test[0, :].shape

(108,)

In [0]:
result = wide_n_deep.model.predict([x_predict_test, x_predict_test])

In [0]:
print('result predicted:', result[0][0])
print('result predicted:', round(result[0][0]))
print('result real:', y_predict_test)

result predicted: 0.00029306783
result predicted: 0.0
result real: 0


In [0]:
result

array([[0.00029307]], dtype=float32)

In [0]:
# prediction for individual
x_predict_test2 = x_test[0:100, :]
y_predict_test2 = y_test[0:100]

In [0]:
results = wide_n_deep.model.predict([x_predict_test2, x_predict_test2])

In [0]:
num_predictions = 100
diff = 0
answer = ""

for i in range(num_predictions):
    val = results[i]
    # print(df.iloc[i])
    
    if ((round(val[0]) - y_predict_test2[i]) == 0):
      answer = "정답"
    else:
      answer = "오답"

    print('Num : ',i+1,' - Predicted: ', int(round(val[0])), 'Actual: ', y_predict_test2[i], '  결과 : '+answer+'\n')
  

Num :  1  - Predicted:  0 Actual:  0   결과 : 정답

Num :  2  - Predicted:  0 Actual:  0   결과 : 정답

Num :  3  - Predicted:  1 Actual:  1   결과 : 정답

Num :  4  - Predicted:  1 Actual:  1   결과 : 정답

Num :  5  - Predicted:  0 Actual:  0   결과 : 정답

Num :  6  - Predicted:  0 Actual:  0   결과 : 정답

Num :  7  - Predicted:  0 Actual:  0   결과 : 정답

Num :  8  - Predicted:  1 Actual:  1   결과 : 정답

Num :  9  - Predicted:  0 Actual:  0   결과 : 정답

Num :  10  - Predicted:  0 Actual:  0   결과 : 정답

Num :  11  - Predicted:  1 Actual:  1   결과 : 정답

Num :  12  - Predicted:  1 Actual:  0   결과 : 오답

Num :  13  - Predicted:  0 Actual:  0   결과 : 정답

Num :  14  - Predicted:  0 Actual:  0   결과 : 정답

Num :  15  - Predicted:  0 Actual:  1   결과 : 오답

Num :  16  - Predicted:  1 Actual:  1   결과 : 정답

Num :  17  - Predicted:  0 Actual:  0   결과 : 정답

Num :  18  - Predicted:  0 Actual:  0   결과 : 정답

Num :  19  - Predicted:  0 Actual:  0   결과 : 정답

Num :  20  - Predicted:  0 Actual:  1   결과 : 오답

Num :  21  - Predicted:  1 Ac